# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data)
print(data.isna().sum())
print('Уникальные значения children:',data['children'].unique())
print('Уникальные значения education:',data['education'].unique())
print('Уникальные значения education_id:',data['education_id'].unique())
print('Уникальные значения family_status:',data['family_status'].unique())
print('Уникальные значения family_status_id:',data['family_status_id'].unique())
print('Уникальные значения gender:',data['gender'].unique())
print('Уникальные значения income_type:',data['income_type'].unique())
print('Уникальные значения debt:',data['debt'].unique())
print('Уникальные значения purpose:',data['purpose'].unique())
print('Количество явных дубликатов:', data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Уникальные значения children: [ 1  0  3  2 -1  4 20  5]
Уникальные значения education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Уникальные значения education_id: [0 1 2 3 4]
Уникальные значения family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальные значения family_status_id: [0 1 2 3 4]
Уникальные значения gender: ['F' 'M' 'XNA']
Уникальные значения income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 

In [3]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Предположим: 

Значение "-1" было переданно с ошибкой и равняется "1" 

Значение "20" было передано с ошибкой и равняется "2"


In [4]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [5]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [6]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Так как человек с неопределенным полом не имеет кредитной задолженности и данное значение всего одно, присвоем ему гендер находящихся в меньшестве.

In [7]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'
data['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

In [8]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

1) С названием столбцов все хорошо.

2) Имеются пропуска в days_employed и total_income.

3) Замечены и исправлены необычные значения в children и gender

4) Столбцец days_employed и total_income тип данных float64, необходимо перевести в int64.

5) При обычном вызове данных, обнаруженно что "пенсионер" имеет положительное значение в days_employed. Так же в глаза бросилось что занчение трудовых дней слишком велико (например №21521), поделил на 365, получил 942 года, поделив на 24 получил 39, вероятней данные указаны в часах. Произвел расчет по данным в строке №21522 (не пенсионер), похоже на правду.

6) Имеется данные в верхнем реестре в столбце education. Исправленно.

7) Имеются дубликаты.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
data_dropna = data.dropna()
#data_dropna.isna().sum()
data['total_income'] = data['total_income'].fillna('change')
data['days_employed'] = data['days_employed'].fillna('change')
#income_type = data['income_type'].unique()
for a in data['income_type'].unique():
    x = data_dropna[data_dropna['income_type'] == a]['total_income'].median() #Посчитали медианное значение total_income
    #для определенного income_type 
    y = data_dropna[data_dropna['income_type'] == a]['days_employed'].median() #Посчитали медианное значение days_employed
    #для определенного income_type 
    data.loc[(data['total_income'] == 'change') & (data['income_type'] == a), 'total_income'] = x 
    data.loc[(data['days_employed'] == 'change') & (data['income_type'] == a), 'days_employed'] = y
    #print('Медианная ЗП', a, 'составляет',x)
    #print('Медианный трудовой стаж', a, 'составляет',y)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null object
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null object
purpose             21525 non-null object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB


In [11]:
display(data[data['total_income'] == 'change'])
data[data['days_employed'] == 'change']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Убедились в том что замена произошла по всем параметрам

**Вывод**

Произведена замена пропусков в total_income и days_employed по медианным значениям соответствующим income_type.

### Замена типа данных

In [12]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null object
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null object
purpose             21525 non-null object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.42,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.32,43,среднее,1,гражданский брак,1,F,компаньон,0,224792,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,156000,сделка с автомобилем
21522,1,-2113.35,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.6,недвижимость
21523,3,-3112.48,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [13]:
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = data.loc[data['income_type'] == 'пенсионер', 'days_employed']/24
data['days_employed'] = data['days_employed'].abs()
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

Заменил значения в столбце days_employed, по наблюдениям указанным в 1 шаге (перевел значения для пенсионеров в дни, все значения сделал положительными). Поменял тип данных в столбцах days_employed и total_income на int64.

### Обработка дубликатов

In [14]:
data.duplicated().sum()

71

In [15]:
data=data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

Во время изучения данных, мною были приведены в порядок знаечния в столбцах education, children, gender. При этом количество явных дубликатов увеличилось с 54 до 71. После этого были удалены дубликаты.

### Лемматизация

In [16]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
purpose_unique = data['purpose'].unique()
aims = ('жилье', "автомобиль", "образование", "свадьба", "недвижимость")
for a in purpose_unique:
    lemmas = ''.join(m.lemmatize(a))
    #print(lemmas)
    for aim in aims:
        if aim in lemmas:
            #data.loc[data['purpose'] == a, 'purpose_group'] = aim
            data.loc[data['purpose'] == a, 'purpose'] = aim
        #lemmas = ''.join(m.lemmatize(a))
        #print(lemmas)
#data.loc[data['purpose'] == 'жилье', 'purpose_group'] = 'недвижимость'
#print(data['purpose_group'].unique()) Пробовал вывести в отдельный столбец, но решил заменить в основном, для определения наличия дубликатов
data.loc[data['purpose'] == 'жилье', 'purpose'] = 'недвижимость'
print(data['purpose'].unique())
data.duplicated().sum()


['недвижимость' 'автомобиль' 'образование' 'свадьба']


334

Изучив лематизированные значения purpose, составил список из основных целей: ('жилье', "автомобиль", "образование", "свадьба", "недвижимость"). После собрал воедино операции с жильем и недвижемостью в одну группу - "недвижимость"

Для повторной проверки на дубликаты, присвоил лематизированные значения в имющийся столбец данных. Появились новые дубликаты, провел отчистку


In [17]:
data=data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

In [18]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21115,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость
21116,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21117,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21118,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


**Вывод**

С помощью лематизации данных удалось выявить основные цели на который люди берут кредит. После замены значений в столбце purpose на выявленные основные цели кредита, появились новые дубликаты, была произведена повторная обработка.  

### Категоризация данных

В данных имеются категории education, family_status, gender и income_type.

По количество детей можно категоризовать в группы: без детей, до 2 детей, многодетные.  

По уровню дохода можно классифицировать: низкий, средний, высокий

In [19]:
education_group = data[['education','education_id']].drop_duplicates().reset_index(drop=True).sort_values('education_id')
education_group

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [20]:
data['education'].value_counts()

среднее                14876
высшее                  5212
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [21]:
family_status_group = data[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True).sort_values('family_status_id')
family_status_group

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [22]:
data['family_status'].value_counts()

женат / замужем          12076
гражданский брак          4124
Не женат / не замужем     2784
в разводе                 1193
вдовец / вдова             943
Name: family_status, dtype: int64

In [23]:
data['purpose'].value_counts()

недвижимость    10578
автомобиль       4272
образование      3964
свадьба          2306
Name: purpose, dtype: int64

In [24]:
def group_master(column):
    if column < x:
        return a
    if column < y :
        return b
    else:
        return c 


Создал функцию для групировки значений на 3 группы: a,b,c - название групп. x,y -верхнее и нижнее значение средней группы соответственно

In [25]:
data['children'].value_counts()

0    13829
1     4798
2     2115
3      329
4       40
5        9
Name: children, dtype: int64

In [26]:
a = 'нет детей'
b = 'до 2х детей'
c = 'многодетные'
x = 1
y = 3
data['children_group'] = data['children'].apply(group_master)
data['children_group'].value_counts()

нет детей      13829
до 2х детей     6913
многодетные      378
Name: children_group, dtype: int64

Сгруппировал по количеству детей. Для выявления зависимости можно будет провести анализ по начальным данным и по сгруппированным и выбрать наилучший результат.

In [27]:
a = 'низкий'
b = 'средний'
c = 'высокий'
x = 100000
y = 200000
data['total_income_group'] = data['total_income'].apply(group_master)
total_income_group = data['total_income_group'].drop_duplicates().reset_index(drop=True)
print(data['total_income_group'].value_counts())
total_income_group

средний    11590
высокий     5067
низкий      4463
Name: total_income_group, dtype: int64


0    высокий
1    средний
2     низкий
Name: total_income_group, dtype: object

По версии банка Credit Suisse.
Один из крупнейших банковских конгломератов в своем ежегодном докладе Global Wealth Report — о благосостоянии населения разных стран — закономерно уделяет внимание и положению среднего класса.

Чтобы попасть в его число, в 2018 году, по версии Credit Suisse, россияне должны были зарабатывать в год $5–25 тыс., то есть по нынешнему курсу (73,35) 30562–152812 рублей в месяц.

In [28]:
data['total_income_ht'] = data['total_income'] //100000
data['total_income_ht'].value_counts()

1     11590
0      4463
2      3584
3       954
4       307
5       113
6        48
7        18
8        12
10        9
9         6
12        4
11        3
17        3
15        2
22        2
14        1
13        1
Name: total_income_ht, dtype: int64

Проверил распределение уровня з/п кратное 100000, решил поделить данные на группы до 100000, от 100000 до 200000, и 200000 и более. Если группировать по версии банка Credit Suisse, группа с низким уровнем дохода включает всего 25 человек.


**Вывод**

В изначальных данных уже было сгруппированы данные по уровню образования и семейному положению. Для ответы на вопросы в шаге 3 добавил группировку по целям кредита, количеству детей (буду анализировать в двух вариантах), по уровню дохода. 

In [31]:
# Код ревьюера
data.groupby('total_income_group').agg(число_клиентов=('total_income','count'),std=('total_income','std'))

,число_клиентов,std
total_income_group,,
высокий,5067,122477.712408
низкий,4463,16682.951725
средний,11590,26561.530110


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
children_group_pivot = data.pivot_table(index='children_group', values='debt', aggfunc=['sum','count'])
try:
    children_group_pivot['ratio'] = children_group_pivot['sum','debt']/children_group_pivot['count','debt']
    display(children_group_pivot.sort_values('ratio'))
except:
    print('Данной группы нет в данных', children_group_pivot[children_group_pivot['count','debt'] == 0])

,sum,count,ratio
,debt,debt,
children_group,,,
нет детей,1061,13829,0.076723
многодетные,31,378,0.082011
до 2х детей,647,6913,0.093592


In [40]:
# Код ревьюера

with pd.option_context('display.float_format','{:.2%}'.format):
    display(children_group_pivot)

,sum,count,ratio
,debt,debt,
children_group,,,
до 2х детей,647,6913,9.36%
многодетные,31,378,8.20%
нет детей,1061,13829,7.67%


In [33]:
children_pivot = data.pivot_table(index='children', values='debt', aggfunc=['sum','count'])
try:
    children_pivot['ratio'] = (children_pivot['sum','debt']/children_pivot['count','debt'])
    display(children_pivot.sort_values('ratio'))
except:
    print('Данной группы нет в данных', children_pivot[children_pivot['count','debt'] == 0])

,sum,count,ratio
,debt,debt,
children,,,
5,0,9,0.000000
0,1061,13829,0.076723
3,27,329,0.082067
1,445,4798,0.092747
2,202,2115,0.095508
4,4,40,0.100000


**Вывод**

Группа клиентов не имеющих дитей, имеют самый низкий процент с задолженностью. Самый высокий процент у группы с 1 и 2 детьми (при просмотре их по отдельности у них примерно одинаковый процент). При детальном разборе выделяются клиенты из группы "многодетные" которые имеют 4 ребенка, у них процент людей с задолженностью самый высокий. Группа с 5 детьми вообще не имеет задолженностей, но и людей в этой группе мало.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
family_status_pivot = data.pivot_table(index='family_status', values = 'debt', aggfunc = ['sum', 'count'])
try:
    family_status_pivot['ratio'] = family_status_pivot['sum','debt']/family_status_pivot['count','debt']
    display(family_status_pivot.sort_values('ratio'))
except:
    print('Данной группы нет в данных', family_status_pivot[family_status_pivot['count','debt'] == 0])
    

,sum,count,ratio
,debt,debt,
family_status,,,
вдовец / вдова,63,943,0.066808
в разводе,85,1193,0.071249
женат / замужем,929,12076,0.076929
гражданский брак,388,4124,0.094083
Не женат / не замужем,274,2784,0.098420


**Вывод**

Вдовец / вдова имеют лучшую платежеспособность. Не женат / не замужем самую худшую. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
total_income_pivot = data.pivot_table(index='total_income_group', values = 'debt', aggfunc = ['sum', 'count'])
try:
    total_income_pivot['ratio'] = total_income_pivot['sum','debt']/total_income_pivot['count','debt']
    display(total_income_pivot.sort_values('ratio'))
except:
    print('Данной группы нет в данных', total_income_pivot[total_income_pivot['count','debt'] == 0])

,sum,count,ratio
,debt,debt,
total_income_group,,,
высокий,358,5067,0.070653
низкий,354,4463,0.079319
средний,1027,11590,0.088611


In [36]:
total_income_ht_pivot = data.pivot_table(index='total_income_ht', values = 'debt', aggfunc = ['sum', 'count'])
try:
    total_income_ht_pivot['ratio'] = total_income_ht_pivot['sum','debt']/total_income_ht_pivot['count','debt']
    display(total_income_ht_pivot.sort_values('ratio'))
except:
    print('Данной группы нет в данных', total_income_ht_pivot[total_income_ht_pivot['count','debt'] == 0])

,sum,count,ratio
,debt,debt,
total_income_ht,,,
17,0,3,0.000000
9,0,6,0.000000
11,0,3,0.000000
12,0,4,0.000000
13,0,1,0.000000
14,0,1,0.000000
15,0,2,0.000000
5,6,113,0.053097


<div class="alert alert-danger">
<h3>❌Комментарий реьюера</h3>
    
Здесь некоторые группы совсем маленькие, так что лучше пользоваться `total_income_group`
</div>    
    

**Вывод**

Группа людей с высоким уровнем доходов имеет меньший процент с задолженностью, однако, если посмотреть на группы с диапазоном з/п в 100000 рублей, среди высокого уровня есть 3 подгруппы (с маленьким количеством людей в группах) с высоким показателем наличия задолженности, это диапазоны 1000000-1100000, 800000-900000 и 2200000-2300000. 

Группа людей со средним уровням дохода, чаще всех имеет задолженность.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
purpose_pivot = data.pivot_table(index='purpose', values = 'debt', aggfunc = ['sum', 'count'])
try:
    purpose_pivot['ratio'] = purpose_pivot['sum','debt']/purpose_pivot['count','debt']
    display(purpose_pivot.sort_values('ratio'))
except:
    print('Данной группы нет в данных', purpose_pivot[purpose_pivot['count','debt'] == 0])


,sum,count,ratio
,debt,debt,
purpose,,,
недвижимость,781,10578,0.073832
свадьба,186,2306,0.080659
образование,370,3964,0.093340
автомобиль,402,4272,0.094101


**Вывод**

Группы клиентов, которые берут кредит на недвижемость, имеют самый низкий процент клиентов с задолженностью, а те кто берет кредит на автомобиль, наоборот с самым высоким процентом.

## Шаг 4. Общий вывод

In [38]:
final_pivot = data.pivot_table(index=['family_status','children_group'], values = 'debt', aggfunc = ['sum', 'count'])
try:
    final_pivot['ratio'] = final_pivot['sum','debt']/final_pivot['count','debt']
    display(final_pivot.sort_values('ratio', ascending=True))    
except:
    print('Данной группы нет в данных', final_pivot[final_pivot['count','debt'] == 0])


sum count     ratio
                                     debt  debt          
family_status         children_group                     
вдовец / вдова        многодетные       0     7  0.000000
                      нет детей        53   831  0.063779
в разводе             нет детей        55   782  0.070332
женат / замужем       многодетные      20   283  0.070671
                      нет детей       514  7272  0.070682
в разводе             до 2х детей      29   399  0.072682
                      многодетные       1    12  0.083333
гражданский брак      нет детей       229  2708  0.084564
женат / замужем       до 2х детей     395  4521  0.087370
Не женат / не замужем нет детей       210  2236  0.093918
вдовец / вдова        до 2х детей      10   105  0.095238
гражданский брак      до 2х детей     151  1350  0.111852
Не женат / не замужем до 2х детей      62   538  0.115242
гражданский брак      многодетные       8    66  0.121212
Не женат / не замужем многодетные       2    10  0.200000

"Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок"

По выведенной информации можно сделать вывод: самая хорошая платежоспособность у людей с семейным положением "вдовец/вдова" и "в разводе" и с количеством детей "нет детей" и "многодетные". Самая плохая платежеспособность у людей с семейным положением "Не женат / не замужем" и "гражданский брак" с количеством детей "многодетные" и "до 2х"детей.

Комбинации семейного положения с количеством детей "до 2х детей" находятся ,в основном, в нижней половине таблице, так же как семейное положение "Не женат / не замужем" и "гражданский брак". 

Семейное положение "в разводе" показывает хорошую платежеспособность не смотря на наличие детей. Так же хорошую платежеспособность показывает нанличие детей - "нет детей".


In [39]:
all_final_pivot = data.pivot_table(index=['education', 'gender', 'income_type', 'family_status','total_income_group','children_group','purpose'], values = 'debt', aggfunc = ['sum', 'count'])
x = len(data)*0.001 #задаем минимальное количество людей в группе
try:
    all_final_pivot['ratio'] = all_final_pivot['sum','debt']/all_final_pivot['count','debt']
    print('Группы клиентов с самым высокой платежеспособностью')
    display(all_final_pivot[all_final_pivot['count','debt'] > x].sort_values('ratio', ascending=True).head(10))
    print('Группы клиентов с самым низкой платежеспособностью')
    display(all_final_pivot[all_final_pivot['count','debt'] > x].sort_values('ratio', ascending=False).head(10))
except:
    print('Данной группы нет в данных', all_final_pivot[final_pivot['count','debt'] == 0])


Группы клиентов с самым высокой платежеспособностью


sum  \
                                                                                                  debt   
education gender income_type family_status         total_income_group children_group purpose             
среднее   F      госслужащий женат / замужем       средний            нет детей      автомобиль      0   
высшее    F      сотрудник   гражданский брак      высокий            нет детей      свадьба         0   
среднее   F      сотрудник   вдовец / вдова        низкий             нет детей      недвижимость    0   
высшее    F      сотрудник   женат / замужем       высокий            нет детей      автомобиль      0   
                                                                                     образование     0   
среднее   F      сотрудник   Не женат / не замужем низкий             нет детей      образование     0   
высшее    M      компаньон   Не женат / не замужем средний            нет детей      недвижимость    0   
                             женат / замужем       высокий            до 2х детей    автомобиль      0   
          F      сотрудник   гражданский брак      средний            до 2х детей    свадьба         0   
          M      компаньон   женат / замужем       высокий            нет детей      недвижимость    0   

                                                                                                  count  \
                                                                                                   debt   
education gender income_type family_status         total_income_group children_group purpose              
среднее   F      госслужащий женат / замужем       средний            нет детей      автомобиль      27   
высшее    F      сотрудник   гражданский брак      высокий            нет детей      свадьба         33   
среднее   F      сотрудник   вдовец / вдова        низкий             нет детей      недвижимость    26   
высшее    F      сотрудник   женат / замужем       высокий            нет детей      автомобиль      22   
                                                                                     образование     27   
среднее   F      сотрудник   Не женат / не замужем низкий             нет детей      образование     23   
высшее    M      компаньон   Не женат / не замужем средний            нет детей      недвижимость    26   
                             женат / замужем       высокий            до 2х детей    автомобиль      26   
          F      сотрудник   гражданский брак      средний            до 2х детей    свадьба         42   
          M      компаньон   женат / замужем       высокий            нет детей      недвижимость    71   

                                                                                                  ratio  
                                                                                                         
education gender income_type family_status         total_income_group children_group purpose             
среднее   F      госслужащий женат / замужем       средний            нет детей      автомобиль     0.0  
высшее    F      сотрудник   гражданский брак      высокий            нет детей      свадьба        0.0  
среднее   F      сотрудник   вдовец / вдова        низкий             нет детей      недвижимость   0.0  
высшее    F      сотрудник   женат / замужем       высокий            нет детей      автомобиль     0.0  
                                                                                     образование    0.0  
среднее   F      сотрудник   Не женат / не замужем низкий             нет детей      образование    0.0  
высшее    M      компаньон   Не женат / не замужем средний            нет детей      недвижимость   0.0  
                             женат / замужем       высокий            до 2х детей    автомобиль     0.0  
          F      сотрудник   гражданский брак      средний            до 2х детей    свадьба        0.0  
          M      компаньон   женат / замужем       высок

Группы клиентов с самым низкой платежеспособностью


sum  \
                                                                                                  debt   
education gender income_type family_status         total_income_group children_group purpose             
среднее   M      сотрудник   гражданский брак      средний            нет детей      автомобиль      7   
                             Не женат / не замужем средний            нет детей      образование    15   
                             гражданский брак      средний            до 2х детей    недвижимость    7   
                             в разводе             средний            до 2х детей    недвижимость    5   
          F      сотрудник   гражданский брак      средний            до 2х детей    автомобиль      9   
          M      сотрудник   Не женат / не замужем средний            до 2х детей    недвижимость    7   
          F      сотрудник   Не женат / не замужем средний            до 2х детей    образование     5   
                 компаньон   гражданский брак      средний            нет детей      образование     5   
высшее    F      пенсионер   женат / замужем       высокий            нет детей      недвижимость    6   
среднее   M      сотрудник   Не женат / не замужем средний            нет детей      автомобиль      9   

                                                                                                  count  \
                                                                                                   debt   
education gender income_type family_status         total_income_group children_group purpose              
среднее   M      сотрудник   гражданский брак      средний            нет детей      автомобиль      24   
                             Не женат / не замужем средний            нет детей      образование     53   
                             гражданский брак      средний            до 2х детей    недвижимость    27   
                             в разводе             средний            до 2х детей    недвижимость    22   
          F      сотрудник   гражданский брак      средний            до 2х детей    автомобиль      40   
          M      сотрудник   Не женат / не замужем средний            до 2х детей    недвижимость    32   
          F      сотрудник   Не женат / не замужем средний            до 2х детей    образование     23   
                 компаньон   гражданский брак      средний            нет детей      образование     23   
высшее    F      пенсионер   женат / замужем       высокий            нет детей      недвижимость    30   
среднее   M      сотрудник   Не женат / не замужем средний            нет детей      автомобиль      48   

                                                                                                      ratio  
                                                                                                             
education gender income_type family_status         total_income_group children_group purpose                 
среднее   M      сотрудник   гражданский брак      средний            нет детей      автомобиль    0.291667  
                             Не женат / не замужем средний            нет детей      образование   0.283019  
                             гражданский брак      средний            до 2х детей    недвижимость  0.259259  
                             в разводе             средний            до 2х детей    недвижимость  0.227273  
          F      сотрудник   гражданский брак      средний            до 2х детей    автомобиль    0.225000  
          M      сотрудник   Не женат / не замужем средний            до 2х детей    недвижимость  0.218750  
          F      сотрудник   Не женат / не замужем средний            до 2х детей    образование   0.217391  
                 компаньон   гражданский брак      средний            нет детей      образование   0.217391  
высшее    F      пенсионер   женат / замужем       высокий            нет детей      недвижимость  0.200000  
среднее 

По отдельно рассмотренным группам самый неплатежноспособными оказались: цель - автомобиль, уровень дохода - средний, семейное положение -  не женат / не замужем, наличие дитей - до 2х детей.

Самы платежеспособные: цель - недвижемость, уровень дохода - высокий, семейное положение -  вдовец / вдова, наличие дитей - нет детей.

Возьмем группы клиентов включающие в себя хотя бы 0,1% от всех клентов в предоставленной базе. Если посмотреть на 10 лучших и худших по платежеспособности групп, видно что и лучший и худший включает в себя показатели которые, прогнозируют как плохую так и хорошую платежеспособность, а первое и последнее место имеют одинаковые занчения в цели, уровне дохода и наличие детей. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.